# ILS - Iterated Local Search

<h3>Classe para Leitura das Instâncias

In [3]:
import sys
import time

import numpy as np



class Instance:
    def __init__(self, filename):

        path = f"{filename}"

        f = open(path, "r")

        self.num_items, self.num_forfeits_pairs, self.budget = map(
            int, f.readline().split(" ")
        )

        f.close()

        # items definition
        items = []
        for i in range(self.num_items):
            items.append(i)

        self.items = np.array(items)

        line_counter = 1

        self.profits = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1
        
        self.weights = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=1
        )
        line_counter += 1

        max = 2 * self.num_forfeits_pairs

        self.forfeit_cost_and_forfeits_pairs = np.loadtxt(
            path, delimiter=" ", skiprows=line_counter, max_rows=max, usecols=(0, 1)
        ).tolist()

        self.forfeits_costs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 == 0
        ]

        self.forfeits_pairs = [
            v for i, v in enumerate(self.forfeit_cost_and_forfeits_pairs) if i % 2 != 0
        ]

        # remove the first element of the sublist
        for i in range(len(self.forfeits_costs)):
            self.forfeits_costs[i].pop(0)

        # transform list of lists into a single list - flatten
        self.forfeits_costs = [
            item for sublist in self.forfeits_costs for item in sublist
        ]

        self.forfeits_costs = np.array(self.forfeits_costs)
        self.forfeits_pairs = np.array(self.forfeits_pairs)      

<h2>Montando a Matriz de Penalidades

In [4]:
def calculate_penalty(items,forfeits_pairs, forfeits_costs):
    mD=np.zeros((int(len(items)),int(len(items))))       
    
    for index, pair in enumerate(forfeits_pairs):
        mD[int(pair[0]),int(pair[1])]=forfeits_costs[index]                
    return mD

<h1>Algoritmo Construtivo (Guloso, Aleatório e Semi-Guloso)</h1>

In [5]:
import random as rd

def greedyalgorithm(items, weights, profits, budget, forfeits, forfeits_costs, alpha,mD):    
    
    cap=budget    

    #print(sorted_items)    
    solution=[]
    scost=[]
    sweights=[]
    cost=0
    index=0
    
    if alpha == 0:  # totalmente aleatório 
        remaining_items=zip(items,weights,profits)
        sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
        #print(tuple(items))        
        
        rd_index=rd.choice(range(0,len(sorted_items))) 
        candidate=sorted_items[rd_index][0]
        
        while cap-weights[candidate]>0:                                        
                cap=cap-weights[candidate]
                solution.append(candidate) 
                
                
                penalidade=sum(mD[candidate][solution])                
                #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                    
                cost=cost+profits[candidate]-penalidade
                
                scost.append(profits[candidate]-penalidade)
                
                sweights.append(weights[candidate])
                
                sorted_items.remove(sorted_items[rd_index])
                rd_index=rd.choice(range(0,len(sorted_items)))  
                candidate=sorted_items[rd_index][0]
        
                    
    else:
            if alpha == 1: #totalmente guloso
                remaining_items=zip(items,weights,profits)
                #print(tuple(items))
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                #sorted_items=sorted(remaining_items, key= lambda x:x[2]/x[1], reverse=True)
                #print(sorted_items)
                candidate=sorted_items[0][0]
                while cap-weights[candidate]>0:                                                  
                    
                    #calcula a nova penalidade para o item a ser inserido no conjunto e reordena                                      
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]/x[1])-sum(mD[x[0]][solution]), reverse=True)                                        
                    
                    cap=cap-weights[candidate]
                    solution.append(candidate)  
                    
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[0][0]} é {penalidade}")
                    
                    #cost=cost+sorted_items[0][2]
                    cost=cost+profits[candidate]-penalidade
                    #cost=cost+sorted_items[index][2]-sum(mD[sorted_items[index][0]][solution])
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                    
                    #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                    sorted_items.remove(sorted_items[0])
                    
                    if len(sorted_items)>=0:
                        #sorted_items=sorted(sorted_items, key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        
                        candidate=sorted_items[0][0]
                    #sorted_items.remove(sorted_items[index])
                    #index=index+1  
                    
            else: # semi-guloso
                remaining_items=zip(items,weights,profits,profits/weights)
                sorted_items=sorted(remaining_items, key= lambda x:x[2]/(x[1]+1), reverse=True)
                
                
                # o cálculo dos limites da lcr
                hmax=sorted_items[0][3]
                hmin=sorted_items[-1][3]

                    #ub = hmax + alpha * (hmin - hmax)
                lb = hmin                            
                ub=hmax + alpha * (hmin - hmax)                  

                    #lista restrita de candidatos
                    #lcr=list(filter(lambda x: x[3]<=ub and x[3] >= lb, sorted_items))                                
                lcr=list(filter(lambda x: x[3]>=ub, sorted_items))
                rd_index=rd.choice(range(0,len(lcr))) 
                candidate=lcr[rd_index][0]
                
                #print(candidate)
                while cap-weights[candidate]>0:  
                                                                          
                    cap=cap-weights[candidate]
                    solution.append(candidate)    
                    #print(f" Solution: {solution}")
                                        
                    
                    #penalidade
                    penalidade=sum(mD[candidate][solution])                
                    #if penalidade>0: print(f"Penalidade do Item {sorted_items[rd_index][0]} é {penalidade}")
                                        
                    cost=cost+profits[candidate]-penalidade                                        
                    
                    scost.append(profits[candidate]-penalidade)
                    
                    sweights.append(weights[candidate])
                
                    #removendo o item já inserido
                    sorted_items.remove(sorted_items[sorted_items.index(lcr[rd_index])])                    
               
                                          
                    #sorted_items.sort(key= lambda x:(x[2]-sum(mD[x[0]][solution]))/(x[1]+1), reverse=True)                                        

                     # o cálculo dos limites da lcr
                    hmax=sorted_items[0][3]
                    hmin=sorted_items[-1][3]
                    
                    #ub = hmax + alpha * (hmin - hmax)
                    lb = hmin                            
                    ub=hmax + alpha * (hmin - hmax)
                 
                    #lista restrita de candidatos                                
                    lcr=list(filter(lambda x: x[3]>=ub, sorted_items))

                    #print(f"LCR: {lcr}")
                    rd_index=rd.choice(range(0,len(lcr))) 
                    candidate=lcr[rd_index][0]
                                
    
    return solution,cost,cap,scost,sweights,sorted_items

<h1>Algoritmo de Busca Local

## Teste Busca Local + Construtivo

In [10]:
import random

def get_solution_cost(solution, profits, forfeits_pairs_set, mD) -> int:
  cost = 0

  for item in solution:
      cost += profits[item]

  for i in solution:
    for j in solution:
      if i > j:
        if (i, j) in forfeits_pairs_set or (j,i) in forfeits_pairs_set:
          cost -= mD[i][j]

  return cost


def swap(solution:set, enter_item:int, leave_item:int, weights, budget) -> set:
  solution.remove(leave_item)
  solution.add(enter_item)

  return solution



def local_search(solution, items, weights, profits, solution_cost, budget, forfeits_costs, forfeits_pairs, mD):

    solution_set = set(solution)
    items_set = set(items)

    # convert forfeits pair in a forfeits pair set
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    improved = True
    ls_cost = int()

    # print(f'constructive solution cost: {solution_cost}')

    # solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
    # print(f'busca local solution cost: {solution_cost}')
    best_improvement = solution_cost
    # print(f'initial_solution_cost:{solution_cost}')

    while improved == True:

      improved = False

      # choosing a random element to enter in solution
      # enter_item = random.choice(tuple(remaining_items)) -> antes o item era escolhida aleatoriamente na lista de restantes

      solution_cost = get_solution_cost(solution_set, profits, forfeits_pairs_set, mD)
      # print(f'get_solution_cost:{solution_cost}')
      cost = solution_cost


      # items not in solution yet
      remaining_items = items_set.difference(solution_set)

      solution_weights = 0
      for item in solution_set:
        solution_weights += weights[item]

      # print(f"solution_weights:{solution_weights}")
      # print(f"budget:{budget}")
      temp_weight = 0

      for sol_item in solution_set: # será iterado apenas uma vez por todos itens que não estão na solução
        for item in remaining_items:
            
          # enter_item = item
          # leave_item = sol_item

          temp_weight = solution_weights - weights[sol_item] + weights[item]
          # print(f'temp_weight:{temp_weight}')

          if temp_weight <= budget: #feasibility
            ls_cost = cost - profits[sol_item] + profits[item]

            for i in solution_set: # add penalties of leave item
              if (i, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[i][sol_item]

              if (item, sol_item) in forfeits_pairs_set:
                  ls_cost += mD[item][i]

            for i in solution_set: # subtract penalties of enter item
              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

              if (i, item) in forfeits_pairs_set:
                  ls_cost -= mD[i][item]

            if best_improvement < ls_cost:
              best_improvement = ls_cost
              enter_item = item
              leave_item = sol_item
              improved = True
              print(f'improved: {best_improvement}')

      if improved == True:
        solution_set = swap(solution_set, enter_item, leave_item, weights, budget)    

    return best_improvement, list(solution_set)

In [11]:
# pasta = 'instances/'
# filename = pasta+"500/kpf_1.txt"
# print(filename)
# problem_instance = Instance(filename)

# mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
# start_time = time.time()
# alpha=1 #guloso
# solution_greedy,cost_greedy,cap,scost,sweights,sorted_items = greedyalgorithm(
#     problem_instance.items,
#     problem_instance.weights,
#     problem_instance.profits,
#     problem_instance.budget,
#     problem_instance.forfeits_costs,
#     problem_instance.forfeits_pairs,
#     alpha,
#     mD,
# )

# print(f"Custo construtivo: {cost_greedy}")
# print(f"Solução construtivo: {solution_greedy}")

# cost_ls, solution_ls = local_search(
#     solution_greedy, 
#     problem_instance.items, 
#     problem_instance.weights, 
#     problem_instance.profits, 
#     cost_greedy,
#     problem_instance.budget,
#     problem_instance.forfeits_costs,
#     problem_instance.forfeits_pairs, 
#     mD)

# end_time = time.time()

# wall_time = end_time - start_time


# print(wall_time)
# print(f"Custo Busca Local: {cost_ls}")
# print(f"Solução Busca Local: {solution_ls}")

# ILS - Iterated Local Search

In [12]:
import random

def get_solution_cost(solution, profits, forfeits_pairs_set, mD) -> int:
  cost = 0

  for item in solution:
      cost += profits[item]

  for i in solution:
    for j in solution:
      if i > j:
        if (i, j) in forfeits_pairs_set:
          cost -= mD[i][j]

  return cost

def perturbation(solution:list(), items, weights, budget) -> list():
    # transform solution list to a set
    solution_set = set(solution)
    items = set(items)

    # items not in solution yet
    remaining_items = items.difference(solution_set)

    for i in range(1,41): # change 40 random numbers
      enter_item = random.choice(tuple(remaining_items))      
      leave_item = random.choice(tuple(solution_set))

      solution_weights = 0

      for item in solution_set:
        solution_weights += weights[item]

      temp_weight = solution_weights - weights[leave_item] + weights[enter_item]

      if temp_weight <= budget: #feasibility
        # swap elements
        solution_set.remove(leave_item)
        solution_set.add(enter_item)

    return list(solution_set)
    


def ILS(items, weights, profits, budget, forfeits_costs, forfeits_pairs, alpha, mD):
    # convert forfeits pair in a forfeits pair set
    forfeits_pairs_set = {tuple(x) for x in forfeits_pairs}

    # generate initial solution
    s0,cost0,cap,scost,sweights,sorted_items = greedyalgorithm(
        items, weights, profits, budget, forfeits_costs, forfeits_pairs, alpha, mD)
    
    print(f"cost construtivo: {cost0}")

    #local search
    cost_star, s_star = local_search(s0, items, weights, profits, cost0, budget, forfeits_costs, forfeits_pairs, mD)
    best_cost = cost_star
    print(f"cost busca local: {cost_star}")

    improved = True

    #iteration control
    iter = 0

    while improved == True:
        
        improved = False

        #iteration control
        iter = iter + 1
        print("################")
        print(f"iter: {iter}\n")
        
        # perturbation in solution
        s_line = perturbation(s_star, items, weights, budget)
        s_line_set = set(s_line)

        #calculate cost for s_line
        s_line_cost = get_solution_cost(s_line_set, profits, forfeits_pairs_set, mD)
        print(f"cost s_line:{s_line_cost}")

        # apply local search in solution s'
        cost_star_line, s_star_line = local_search(s_line, items, weights, profits, s_line_cost, budget, forfeits_costs, forfeits_pairs, mD)

        print(f"cost star_line: {cost_star_line}")

        # acceptance criterion
        if cost_star_line > best_cost:
            best_cost = cost_star_line
            improved = True

    return s_star_line, best_cost

## Teste ILS

In [13]:
pasta = 'instances/'
filename = pasta+"500/kpf_1.txt"
print(filename)
problem_instance = Instance(filename)

mD=calculate_penalty(problem_instance.items,problem_instance.forfeits_pairs, problem_instance.forfeits_costs)
start_time = time.time()
alpha=1 #guloso

start_time = time.time()

solution_ils, cost = ILS(problem_instance.items, 
    problem_instance.weights,
    problem_instance.profits,
    problem_instance.budget,
    problem_instance.forfeits_costs,
    problem_instance.forfeits_pairs,
    alpha,
    mD,)

end_time = time.time()

wall_time = end_time - start_time


print(wall_time)
print(f"Custo ILS: {cost}")
print(f"Solução {solution_ils}")


solution_weights = 0
for item in solution_ils:
  solution_weights += problem_instance.weights[item]
print(f"Solution weights: {solution_weights}")


instances/500/kpf_1.txt
cost construtivo: 1767.0
improved: 1769.0
improved: 1773.0
improved: 1775.0
improved: 1777.0
improved: 1780.0
improved: 1787.0
improved: 1809.0
improved: 1830.0
improved: 1855.0
improved: 1856.0
improved: 1857.0
improved: 1860.0
improved: 1870.0
cost busca local: 1870.0
################
iter: 1

cost s_line:1765.0
improved: 1804.0
improved: 1808.0
improved: 1851.0
improved: 1862.0
improved: 1869.0
improved: 1871.0
improved: 1887.0
cost star_line: 1887.0
################
iter: 2

cost s_line:1687.0
improved: 1689.0
improved: 1700.0
improved: 1719.0
improved: 1720.0
improved: 1724.0
improved: 1725.0
improved: 1736.0
improved: 1737.0
improved: 1739.0
improved: 1742.0
improved: 1756.0
improved: 1758.0
improved: 1770.0
improved: 1771.0
improved: 1783.0
improved: 1784.0
improved: 1786.0
improved: 1789.0
improved: 1803.0
improved: 1805.0
cost star_line: 1805.0
16.103509664535522
Custo ILS: 1887.0
Solução [4, 5, 8, 9, 11, 14, 16, 17, 18, 19, 21, 22, 24, 28, 29, 31, 32, 